# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     David Alejandro Rangel Rodríguez|   |
| **Fecha**     5 de febrero del 2026 |   |
| **Expediente*756203* |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

¿Qué puedes observar al ajustar los valores de `alpha`? 

Compara los resultados de los coeficientes utilizando valores diferentes de $\alpha$ y los $R^2$ resultantes.



In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

adver = pd.read_csv('Advertising.csv')

if 'Unnamed: 0' in adver.columns:
    adver = adver.drop(columns=['Unnamed: 0'])

X = adver[['TV', 'radio', 'newspaper']]
y = adver['sales']

#Sin penalización (Regresión Lineal Múltiple)
lr = LinearRegression()
lr.fit(X, y)

print("1. Sin penalización (Linear Regression):")
print(f"R^2: {lr.score(X, y):.4f}")
print(f"Coeficientes: {dict(zip(X.columns, lr.coef_))}")
print("-" * 30)

#MODELOS CON PENALIZACIÓN (Ridge y Lasso)
alphas = [0.1, 1, 10, 100, 1000]

results = []

for a in alphas:
    # 2. Regresión Ridge (L2)
    ridge = Ridge(alpha=a)
    ridge.fit(X, y)
    
    # 3. Regresión Lasso (L1)
    lasso = Lasso(alpha=a)
    lasso.fit(X, y)
    
    results.append({
        'alpha': a,
        'Ridge_R2': ridge.score(X, y),
        'Ridge_Coefs': ridge.coef_,
        'Lasso_R2': lasso.score(X, y),
        'Lasso_Coefs': lasso.coef_
    })

# Resultados comparativos
print(f"{'Alpha':<8} | {'Ridge R2':<10} | {'Lasso R2':<10} | {'Lasso Coefs (TV, Radio, News)':<30}")
for res in results:
    print(f"{res['alpha']:<8} | {res['Ridge_R2']:<10.4f} | {res['Lasso_R2']:<10.4f} | {np.round(res['Lasso_Coefs'], 4)}")

1. Sin penalización (Linear Regression):
R^2: 0.8972
Coeficientes: {'TV': np.float64(0.045764645455397615), 'radio': np.float64(0.18853001691820448), 'newspaper': np.float64(-0.0010374930424763007)}
------------------------------
Alpha    | Ridge R2   | Lasso R2   | Lasso Coefs (TV, Radio, News) 
0.1      | 0.8972     | 0.8972     | [ 0.0458  0.1879 -0.0007]
1        | 0.8972     | 0.8970     | [0.0457 0.1835 0.    ]
10       | 0.8972     | 0.8801     | [0.0448 0.1427 0.    ]
100      | 0.8972     | 0.5615     | [0.0339 0.     0.    ]
1000     | 0.8970     | 0.0000     | [0. 0. 0.]


### ¿Qué puedes observar al ajustar los valores de alpha?

Al incrementar el valor de $\alpha$, se intensifica la penalización sobre la magnitud de los coeficientes, lo que obliga al modelo a reducir su complejidad para evitar el sobreajuste (overfitting). En la regresión Ridge (L2), los coeficientes disminuyen gradualmente hacia cero sin llegar a anularse, manteniendo todas las variables pero suavizando su impacto. En contraste, en Lasso (L1), un $\alpha$ mayor fuerza a los coeficientes de las variables menos relevantes (como newspaper) a ser exactamente cero, funcionando como una herramienta de selección de variables que simplifica el modelo. En ambos casos, mientras más alto es $\alpha$, la varianza disminuye y el sesgo aumenta, lo que se traduce en una caída del $R^2$ en los datos de entrenamiento a cambio de un modelo más generalizable